In [1]:
import numpy as np
import emg3d
import pygimli as pg
from emg3d.inversion.pygimli import Kernel, Inversion

In [2]:
grid = emg3d.TensorMesh([[1, 1], [1, 1], [1, 1, 1]])
model = emg3d.Model(grid, np.arange(1, grid.n_cells+1)*100, mapping='Conductivity')
survey = emg3d.Survey(emg3d.TxElectricPoint((1, 1, 1, 1, 1)), emg3d.RxElectricPoint((3, 3, 3, 3, 3)), 1)
sim = emg3d.Simulation(survey=survey, model=model)

In [3]:
markers = np.zeros(model.shape, order='F', dtype=int)
markers[::2, :, :] = 1
markers[:, ::2, :] = 2
markers[:-1, :, ::2] = 3

In [4]:
fop = Kernel(simulation=sim, markers=markers)
INV = Inversion(fop=fop)

INV.fop.setRegionProperties(2, background=True)
INV.fop.setRegionProperties(3, limits=(20, 30), startModel=25.0)
INV.fop.setRegionProperties(1, single=True, limits=(100, 200), startModel=150)
INV.fop.setRegionProperties(0, fix=True, startModel=999)

In [5]:
model.property_x.ravel()

array([ 100.,  500.,  900.,  300.,  700., 1100.,  200.,  600., 1000.,
        400.,  800., 1200.])

In [6]:
markers.ravel('F')

array([3, 2, 3, 0, 2, 2, 1, 0, 3, 2, 3, 0])

In [7]:
fop.mesh().cellMarkers()

12 [8, 4, 9, 0, 5, 6, 3, 1, 10, 7, 11, 2]

In [8]:
gmodel = fop.model2gimli(model.property_x)
gmodel

array([ 700.,  100.,  300.,  900., 1100.])

In [9]:
fop.model2emg3d(gmodel)

array([[[ 100.,  500.,  900.],
        [ 300.,  700., 1100.]],

       [[ 200.,  600., 1000.],
        [ 999.,  999.,  999.]]])